# Capstone 2: narrative analytics and experimentation

In [23]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

#set the seed to assure you get the same answers, otherwise, you could remove the following line
random.seed(100)

file = 'ab_data.csv'
df = pd.read_csv(file)

In [24]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


# PROBABILITY:

In [25]:
# allows us to see the rows of the dataset 
df.shape[0]

294478

Retrieving the unique number of ID's 

In [26]:
df.user_id.nunique()

290584

The proportion of users converted.

In [27]:
"%.4f" % (df.converted.sum() / df.converted.count())

'0.1197'

Samples on two demonsions grouped by control & treatment and new & old page



In [28]:
df.groupby(['group','landing_page']).count()['converted']

group      landing_page
control    new_page          1928
           old_page        145274
treatment  new_page        145311
           old_page          1965
Name: converted, dtype: int64

Checking for missing values

In [29]:
df.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

create a new dataset where the treatment group is the new_page and the control group is the old_page

In [48]:
df2 = df.query("group == 'control' & landing_page == 'old_page'")

In [49]:
df3 = df.query("group == 'treatment' & landing_page == 'new_page'")

In [50]:
df2.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0
7,719014,2017-01-17 01:48:29.539573,control,old_page,0


In [51]:
df3.head()

,user_id,timestamp,group,landing_page,converted
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


In [53]:
df2.nunique()

user_id         145274
timestamp       145274
group                1
landing_page         1
converted            2
dtype: int64

In [54]:
df3.nunique()

user_id         145310
timestamp       145311
group                1
landing_page         1
converted            2
dtype: int64

check all rows were removed

In [56]:
df3[((df3['group'] == 'treatment') & (df3['landing_page'] == 'new_page')) == False].shape[0]

0

are there any unique id's in the new dataframe

In [59]:
df3.user_id.nunique()

145310

In [60]:
df3[df3.user_id.duplicated() == True]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [61]:
df3.drop_duplicates(subset='user_id', keep='first', inplace=True)

C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [74]:
final_df=pd.concat([df2,df3])

In [76]:
final_df.nunique()

user_id         290584
timestamp       290584
group                2
landing_page         2
converted            2
dtype: int64

In [80]:
final_df.groupby(['group','landing_page']).count()['converted']

group      landing_page
control    old_page        145274
treatment  new_page        145310
Name: converted, dtype: int64

counting the unique id for the concatenated a/b dataset

In [84]:
final_df.user_id.nunique()

290584

Finding out the probability for user whether they are on the new_page or old_page:

In [99]:
"%.4f"% (final_df.converted.sum() / final_df.converted.count())

'0.1196'

Probability for control group if they converted: 

In [100]:
obs_old = final_df.query('group == "control"')['converted'].sum() / final_df.query('group == "control"')['converted'].count()
'%.4f' %obs_old

'0.1204'

Probabiltiy for treatment group if they converted:

In [102]:
obs_new = final_df.query('group == "treatment"')['converted'].sum() / final_df.query('group == "treatment"')['converted'].count()
'%.4f' %obs_new

'0.1188'


The conversion rate for the new treatment was lower than the conversion rate for the whole group. The new treatment rate was also lower than the control group. The results show that there is not sufficient evidence that shows that the new treatment group results in higher conversion rates. 

# A/B Test:

H0: Pold =  Pnew           
H1: Pold <> Pnew



Null hypothesis Pnew and Pold both have success rates equal to the converted success rates 

In [103]:
null_rate = df['converted'].sum() / df.shape[0]
null_rate

0.11965919355605512